This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb

!cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .
!cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_metrics_3d.py .


[jm4669@radio-gpu CTs_CUIMC_New1033_liver]$ du -h -d 1 . | sort -h
0       ./PDS_AUTO_RECIST_METNET2956_D2015_04_27_E2863_CT_S0007_8707
0       ./PDS_AUTO_RECIST_METNET5635_D2013_06_06_E7354_CT_S0204_0841

In [1]:
# coding: utf-8

# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
%reload_ext autoreload
%autoreload 2

import os
import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
from utils_test import *

import cv2
from PIL import Image
import torch.nn.functional as F
from utils.utils import get_confusion_matrix
def convert_name(name):
    new = name.replace('/','_')
    return new

def get_palette(n):
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette
    
# def save_pred(preds, sv_path, name):

#     preds = preds.cpu().numpy().copy()
#     preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
#     for i in range(preds.shape[0]):
#         cv2.imwrite(os.path.join(sv_path, convert_name(name[i])) , preds[i])

        
def save_pred( preds, sv_path, name):
    palette = get_palette(256)
    preds = preds.cpu().numpy().copy()
    preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
    for i in range(preds.shape[0]):
        pred = preds[i]
        save_img = Image.fromarray(pred)
        save_img.putpalette(palette)
        save_img.save(os.path.join(sv_path, convert_name(name[i]) ))
        

def testval_lesion(config, test_dataset, testloader, model,
            sv_dir='', sv_pred=True, device = None):
    model.eval()
    confusion_matrix = np.zeros(
        (config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES))
    with torch.no_grad():
        for index, batch in enumerate(testloader):
            image, label, _, batchname = batch
            size = label.size()
            if device is None:
                image = image.cuda()
                label = label.long().cuda()
            else:
                image = image.to(device)
                label = label.long().to(device)

            pred = model(image)
            if pred.size()[-2] != size[-2] or pred.size()[-1] != size[-1]:
                pred = F.upsample(pred, (size[-2], size[-1]),
                                  mode='bilinear')

            confusion_matrix += get_confusion_matrix(
                label,
                pred,
                size,
                config.DATASET.NUM_CLASSES,
                config.TRAIN.IGNORE_LABEL)

            if sv_pred:
                sv_path = os.path.join(sv_dir, 'test_val_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                save_pred(pred, sv_path, batchname)

            if index % 100 == 0:
                logging.info('processing: %d images' % index)
                pos = confusion_matrix.sum(1)
                res = confusion_matrix.sum(0)
                tp = np.diag(confusion_matrix)
                IoU_array = (tp / np.maximum(1.0, pos + res - tp))
                mean_IoU = IoU_array.mean()
                logging.info('mIoU: %.4f' % (mean_IoU))

    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)
    pixel_acc = tp.sum() / pos.sum()
    mean_acc = (tp / np.maximum(1.0, pos)).mean()
    IoU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IoU = IoU_array.mean()

    return mean_IoU, IoU_array, pixel_acc, mean_acc

In [2]:
!ls -lht ./cache

total 284M
-rw-r--r--. 1 jm4669 domain users 1.4M Jan 20 16:32 inference_gt_roidb.pkl
-rw-r--r--. 1 jm4669 domain users  25M Jan 14 00:12 PDS_AMGEN_20020408_22Cat_test_gt_roidb.pkl
-rw-r--r--. 1 jm4669 domain users  14M Jan 12 22:22 Cougar302_80pts_test_gt_roidb.pkl
-rw-r--r--. 1 jm4669 domain users 214M Jan  7 12:29 CUIMC_New1033_gt_roidb.pkl
-rw-r--r--. 1 jm4669 domain users  31M Jan  6 17:30 CUIMC_20201027_inference_gt_roidb.pkl


In [3]:
cache_path = './cache/'
name = 'CUIMC_New1033'

DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS New 1033 CUIMC cases 2021-12-22.xlsx'


cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')
print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb


# In[25]:

from utils_test import __get_annotation__

sv_dir = mask_name = 'mask_1988'
sv_path = os.path.join(sv_dir, 'test_val_results')

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):
    if i%500==0:
        print('='*40,i)

    one = roidb[i]
    onename = one['image']
    pred_im = Image.open(os.path.join(sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox

Loading cached gt_roidb from %s ./cache/CUIMC_New1033_gt_roidb.pkl


/mnt/fast-data/mjc/envs/Lesions/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


======================================== 0
======================================== 500
======================================== 1000
======================================== 1500
======================================== 2000
======================================== 2500
======================================== 3000
======================================== 3500
======================================== 4000
======================================== 4500
======================================== 5000
======================================== 5500
======================================== 6000
======================================== 6500
======================================== 7000
======================================== 7500
======================================== 8000
======================================== 8500
======================================== 9000
======================================== 9500
======================================== 10000
=====================================

22

In [5]:
# site_list = range(cfg.DATASET.NUM_CLASSES) #8 is liver
site_list = [8]
D_CT = {} #contain 1:roidb(gt from cache by MVPNet)
        #2:predicted bboxes
        #3:predicted segmentations
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]

In [6]:
# In[37]:

import pandas as pd
df = pd.read_excel(DL_info)

def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        df[col] = df[col].str.replace(ori,new, case = False) 
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instence should be str or list')

pd_str_replace(df, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/W-drive")
pd_str_replace(df, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw/PDS_AUTO_RECIST", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToMatlab", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")


# In[52]:

sys.path.append('/mnt/fast-data/mjc/DeepLesion/Codes/CollectData/')

from print_weasis_raw_data import WeasisRawDataPrinter
from read_weasis_raw_data import WeasisRawFileReader


import os
import numpy as np
import pydicom
import matplotlib.pylab as plt


data_path = ""
metadata_path = "/mnt/fast-data/mjc/AutoRECIST/Outputs/Dicom_header"
if not os.path.exists(metadata_path):
    os.mkdir(metadata_path)


D_dir2header_df = {}


def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:
                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [7]:
from utils_test import compare_site
from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():

    if not compare_site(row.Location , site_list):
        # only calculate one lesion site
        continue
    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT and skip it')
        continue

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
    Metrics_vol.extend(hit)
    
    _n = len(Metrics_vol)
    if _n%100==0 or _n<10 or _n in [20,50]:
        print(_, _n)
        df_metrics = pd.DataFrame(Metrics_vol, 
                                  columns = ['file_name','igt','merge','#gt','#pred',
                                             'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                             'area_gt','area_pred','intersection','union']) 
        df_metrics.to_csv('temp_%s_.csv'%'ScaleNAS_Q5_9Slices_CUIMC_New1033')

5 1
6 2
7 3
13 4
19 5
21 6
26 7
27 8
28 9
59 20
307 50
494 100
875 200
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 not in D_CT and skip it
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872 n

In [8]:
len(Metrics_vol)

1115

In [9]:
print(name)
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

CUIMC_New1033


,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000,1115.000
mean,1.004,0.000,1.000,10.457,0.337,0.459,6.119,0.308,11933.391,25249.908,7610.237,29573.062
std,0.067,0.000,0.000,7.599,0.224,0.272,37.800,0.342,54686.614,88648.140,32371.271,99468.464
min,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,12.000,0.000,0.000,12.000
5%,1.000,0.000,1.000,2.000,0.000,0.000,0.000,0.000,97.000,0.000,0.000,165.400
25%,1.000,0.000,1.000,5.000,0.130,0.230,0.176,0.041,365.000,444.000,192.000,736.000
50%,1.000,0.000,1.000,10.000,0.371,0.541,0.634,0.148,1070.000,1662.000,696.000,2267.000
75%,1.000,0.000,1.000,13.000,0.514,0.679,1.621,0.490,4052.500,7714.000,2515.500,9759.500
95%,1.000,0.000,1.000,25.000,0.679,0.809,22.598,1.000,45455.800,90544.000,30749.400,127851.100
max,2.000,0.000,1.000,72.000,0.819,0.901,978.292,1.000,977063.000,718051.000,431193.000,985628.000


In [10]:
df_metrics[df_metrics.dice_score>0.25].describe([.5])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,827.000,827.000,827.000,827.000,827.000,827.000,827.000,827.000,827.000,827.000,827.000,827.000
mean,1.002,0.000,1.000,10.547,0.444,0.599,0.994,0.208,14364.591,16906.475,9785.289,21485.778
std,0.049,0.000,0.000,7.885,0.152,0.149,1.035,0.217,62779.224,60360.638,37280.454,80213.086
min,1.000,0.000,1.000,1.000,0.143,0.250,0.000,0.000,26.000,14.000,12.000,62.000
50%,1.000,0.000,1.000,10.000,0.449,0.620,0.659,0.128,1198.000,1793.000,974.000,2199.000
max,2.000,0.000,1.000,72.000,0.819,0.901,5.869,0.850,977063.000,615132.000,431193.000,985628.000


In [11]:

for col in df.columns.tolist():
    print(col , len(set(df[col].tolist() )) )


PatientID 1033
PatientName 1033
StudyDate 681
User ID 3
Tumor QA State 1
Tumor Label 46
Location 21
Uni 4931
Perp 4670
Bi 5266
Volume 5290
Update Date Time 5327
Raw File Name 5327
Tumor Instance UID 5327
Image File Path 1338
Contour File Path 5327


In [12]:
dfinnermerge = pd.merge(df_metrics,df,how='inner',left_on='file_name' , right_on='Contour File Path')

for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )



pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)
print("="*80)
print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

file_name 1115
igt 2
merge 1
#gt 1
#pred 25
iou_score 988
dice_score 988
over_seg 976
under_seg 921
area_gt 955
area_pred 795
intersection 847
union 1015
PatientID 470
PatientName 308
StudyDate 270
User ID 3
Tumor QA State 1
Tumor Label 28
Location 1
Uni 1084
Perp 1063
Bi 1111
Volume 1113
Update Date Time 1115
Raw File Name 1115
Tumor Instance UID 1115
Image File Path 324
Contour File Path 1115
In total, 324 CT series; 2156 AI detections 
sensitivity is 0.886(988/1115) FP-rate is 3.6 per CT-serie at threshold 0
sensitivity is 0.809(902/1115) FP-rate is 3.9 per CT-serie at threshold 0.1
sensitivity is 0.761(849/1115) FP-rate is 4.0 per CT-serie at threshold 0.2
sensitivity is 0.742(827/1115) FP-rate is 4.1 per CT-serie at threshold 0.25
sensitivity is 0.555(619/1115) FP-rate is 4.7 per CT-serie at threshold 0.5


In [13]:
df_metrics.to_csv('%s_liver_seg_metrics_vol_20220106_reduceFP.csv'%'ScaleNAS_Q5_9Slices_CUIMC_New_1033cases')

In [22]:
try:
    %run -i Save_Image_Masks.py
except:
    pass

site_list to save is  [8]
Save path is /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_New1033_liver/
Skiped /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_New1033_liver/PDS_AUTO_RECIST_METNET2909_D2015_02_02_E1753_CT_S0005_7767
Skiped /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_New1033_liver/PDS_AUTO_RECIST_METNET2909_D2015_02_02_E8642_CT_S0002_4793
Skiped /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_New1033_liver/PDS_AUTO_RECIST_METNET2910_D2015_02_06_E8602_CT_S0005_3151
Skiped /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_New1033_liver/PDS_AUTO_RECIST_METNET2911_D2015_02_13_E1924_CT_S0003_7579
Skiped /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_New1033_liver/PDS_AUTO_RECIST_METNET2911_D2015_02_13_E2523_CT_S0004_1732
Skiped /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1

In [15]:
CUIMC New Testing-set
308 pts, 324 CT series, 1115 liver lesions
2156 AI detections
Sensitivity 74%=(827/1115)
FP rate is 3.6 FPs/CT-series


SyntaxError: invalid syntax (<ipython-input-15-b5523bfe4f08>, line 1)

In [18]:
CT_gts = get_gts(k , df)

/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2956/D2015_04_27/E2863/CT/S0007_8707
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5220/D2014_09_11/E4826/CT/S0002_5530


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
/mnt/fast-disk1/mjc/AutoRecist/Codes/ScaleNAS_9Slices/tools/Save_Image_Masks.py in <module>
     94                     colors = [20,30,255]
     95                     label = ix2labelname(j)
---> 96                     for c in contours:
     97                         c = np.reshape(c,(-1,2))
     98                         if c.shape[0]:

TypeError: 'NoneType' object is not iterable

AttributeError: 'NoneType' object has no attribute 'extend'

In [26]:
D_CT.keys()

dict_keys(['/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2909/D2015_02_02/E1753/CT/S0005_7767', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2909/D2015_02_02/E8642/CT/S0002_4793', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2910/D2015_02_06/E8602/CT/S0005_3151', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2911/D2015_02_13/E1924/CT/S0003_7579', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2911/D2015_02_13/E2523/CT/S0004_1732', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2912/D2015_03_11/E2096/CT/S0005_1035', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2913/D2015_03_12/E6834/CT/S0010_8451', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2913/D2015_03_12/E2189/CT/S0008_1228', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2914/D2015_03_16/E0352/CT/S0005_2011', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET2914/D2015_03_16/E8300/CT/S0002_0406', '/mnt/fas

In [ ]:
stop

In [ ]:
# In[55]:

print(name)
df_miss = pd.DataFrame(Missed_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_miss.describe([.05, .25, .5, .75, .95])

In [24]:
df_image.columns






Index(['ImageName', 'BitsAllocated', 'BitsStored', 'Columns', 'HighBit',
       'ImageOrientationPatient_0', 'ImageOrientationPatient_1',
       'ImageOrientationPatient_2', 'ImageOrientationPatient_3',
       'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
       'ImagePositionPatient_0', 'ImagePositionPatient_1',
       'ImagePositionPatient_2', 'Modality', 'PatientID',
       'PhotometricInterpretation', 'PixelRepresentation', 'PixelSpacing_0',
       'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows',
       'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID',
       'StudyInstanceUID', 'WindowCenter', 'WindowWidth'],
      dtype='object')